# Overview

This notebook provides an overview/abbreviated tutorial of the APTD package.

### Experiment and design

We will use the example of an cognitive experiment, a variant of the *extended Stroop task*.  In this experiment, subjects are presented with stimuli (words on a screen) that vary along two *feature dimensions*: the color of the lettering that spells the word (color), and the sequence of letters itself (word).  Following the presentation of a stimulus, the subject must respond in a prescribed manner.  Like stimuli, responses vary along two two dimensions: manual (pointing) and vocal (speaking).

Each subject selects a response to each stimulus based on which task or tasks they have been assigned.  In this context, a **task** means a mapping from a single simulus dimension to a single response dimension; it is assumed that concrete mappings between the levels of each feature dimension and the levels of each response dimension (e.g., mapping colors to verbal responses) has been specified *a priori*.  Thus, once the experiment begins, the expermentor can specify a task simply by naming an feature dimension and a response dimension (for example, the experimenter may specify "point/color," and from this the subject will know which of several sets of instructions to follow).

\

**Summary of stimulus and response variables**

\

input dimension | name  | symbol  | number of feature values | feature 0 | feature 1 
------------- |------------- | ------------- | ------------- | ------------- |------------- 
0 | color | C | 2 | red (R) | green (G)
1 | word | W | 2 | red (R) | green (G)

output dimension | name  | symbol  | number of feature values | feature 0 | feature 1 
------------- |------------- | ------------- | ------------- | ------------- |------------- 
0 | point | P | 2 | red (R) | green (G)
1 | speak | S | 2 | red (R) | green (G)

\

## Experimental data


The demonstration will use the following (synthetic) data. Rows of the table are ordered such that stimuli follow lexicographic order
  * concretely, this means RR, RG, GR, GG
  * if the color dimension had taken three different feature values, RGB, then the order would be RR, RG, RB, GR, GG, GB
  * thus the entries in the column labeled `input (numpy code)` are identical for every task

\

task | stimulus (dim 0: color)  | stimulus (dim 1: word)  | stimulus (vectorized) | instruction (dim 0: speak) | intruction (dim 1: point) | instruction (vectorized) | subject fmri response (vectorized) | subject behavior response (vectorized)
------------- |------------- | ------------- | ------------- | ------------- |------------- | ------------- | ------------- | ------------- 
name color | R | R | [1, 0, 1, 0] |   R | - | [1, 0, 0, 0] |[1, 0, 0, 0] |[1, 0, 0, 0] |
" | R | G | [1, 0, 0, 1] |   R | - | [1, 0, 0, 0] |[1, 0, 0, 0] |[1, 0, 0, 0] |
" | G | R | [0, 1, 1, 0] |   G | - | [0, 1, 0, 0] |[0, 1, 0, 0] |[0, 1, 0, 0] |
" | G | G | [0, 1, 0, 1] |   G | - | [0, 1, 0, 0] |[0, 1, 0, 0] |[0, 1, 0, 0] |
point color | R | R | [1, 0, 1, 0] |   - | R | [0, 0, 1, 0] |[0, 0, 1, 0] |[0, 0, 1, 0] |
" | R | G | [1, 0, 0, 1] |   - | R | [0, 0, 1, 0] |[0, 0, 1, 0] |[0, 0, 1, 0] |
" | G | R | [0, 1, 1, 0] |   - | G | [0, 0, 0, 1] |[0, 0, 0, 1] |[0, 0, 0, 1] |
" | G | G | [0, 1, 0, 1] |   - | G | [0, 0, 0, 1] |[0, 0, 0, 1] |[0, 0, 0, 1] |
read color | R | R | [1, 0, 1, 0] |   R | - | [1, 0, 0, 0] |[1, 0, 0, 0] |[1, 0, 0, 0] |
" | R | G | [1, 0, 0, 1] |   G | - | [0, 1, 0, 0] |[0, 1, 0, 0] |[0, 1, 0, 0] |
" | G | R | [0, 1, 1, 0] |   R | - | [1, 0, 0, 0] |[1, 0, 0, 0] |[1, 0, 0, 0] |
" | G | G | [0, 1, 0, 1] |   G | - | [0, 1, 0, 0] |[0, 1, 0, 0] |[0, 1, 0, 0] |
point color | R | R | [1, 0, 1, 0] |   - | R | [0, 0, 1, 0] |[0, 0, 1, 0] |[0, 0, 1, 0] |
" | R | G | [1, 0, 0, 1] |   - | G | [0, 0, 0, 1] |[0, 0, 0, 1] |[0, 0, 0, 1] |
" | G | R | [0, 1, 1, 0] |   - | R | [0, 0, 1, 0] |[0, 0, 1, 0] |[0, 0, 1, 0] |
" | G | G | [0, 1, 0, 1] |   - | G | [0, 0, 0, 1] |[0, 0, 0, 1] |[0, 0, 0, 1] |


# Formatting overview : data stored in a dictionary

Use of this library depends on formatting of experimental data.

* most data is broken down by 
  * what is being measured (eg, behavior vs fmri)
  * what task (or set of tasks) is being performed
    * task sets are encoded by a string witn input/output dimensions separated by commas
    * example: '0,0,1,1' means 'point color and read word'
* since the numpy input code is the same for every task, we only input this data once, under keyword "data"

In [ ]:
from importlib import reload

import numpy as np
import aptfuns
reload(aptfuns)
from aptfuns import *

In [ ]:
#   define the dictionary
aptd              =   {}

### size and number of dimensions

In [ ]:
#   record the number of input + output dimensions
aptd['nid']       =   2 # number of input dimensions
aptd['nod']       =   2 # number of output dimensions

#   record the number of input + output dimensions
aptd['id__nu']    =   [2,2] # number of distinct feature values in each input dimension
aptd['od__nu']    =   [2,2] # number of coordinates/entries in each output vector (THIS IS A LITTLE IRRELEVANT FOR ABBY)
aptd['id__u0']    =   np.concatenate(([0],np.cumsum(aptd['id__nu'])[:-1])) # this list records the location of the "first unit" in each input dimension
aptd['od__u0']    =   np.concatenate(([0],np.cumsum(aptd['od__nu'])[:-1])) # this list records the location of the "first unit" in each output dimension

### (sets of) tasks performed

A "task" is stored as a list of length 2: [a,b] represents the task sending input dimension a to output dimension b.  

A "perofrmance list" is a list of tasks.  For example, [[a,b], [a',b']] means "perform tasks [a,b] and [a',b'] simultaneously."  

A "performance list list" is a list of performance lists.  Typically we only use this list for record keeping (so that we can remember which perofrmance lists we have and have not trained the network to perform).  The line below declares that we have data for (1) color pointing, (2) color speaking, (3) word pointing, (4) word speaking, (5) simultaneous color pointing and word speaking, and (6) simultaneous color speaking while word pointing.

In [ ]:
aptd['pll']       =   [[[0,0]], [[0,1]], [[1,0]], [[1,1]], [[0,0],[1,1]], [[0,1],[1,0]]]

### stimuli

We assume that the agent recieves the same set of input stimuli no matter what task is performed.  Moreover, we assume that stimuli are sorted lexicographically according to feature value. (**abby doesn't neet to input this one**)

In [ ]:
aptd['data']      =   np.array(   [ [1, 0, 1, 0],
                                    [1, 0, 0, 1],
                                    [0, 1, 1, 0],
                                    [0, 1, 0, 1]  ]
                                   )

### instructions, behavior, and activity patterns

Record the activity patterns in the hidden layer for each performance list.

In [ ]:
aptd['fmri']          = {}
aptd['fmri']['0,0']   = np.array(   [   [1, 0, 0, 0],
                                        [1, 0, 0, 0],
                                        [0, 1, 0, 0],
                                        [0, 1, 0, 0]  ]
                                   )

aptd['fmri']['0,1']   = np.array(   [   [1, 0, 0, 0],
                                        [1, 0, 0, 0],
                                        [0, 1, 0, 0],
                                        [0, 1, 0, 0]  ]
                                   )

aptd['fmri']['1,0']   = np.array(   [   [0, 0, 1, 0],
                                        [0, 0, 0, 1],
                                        [0, 0, 1, 0],
                                        [0, 0, 0, 1]  ]
                                   )

aptd['fmri']['1,1']   = np.array(   [   [0, 0, 1, 0],
                                        [0, 0, 0, 1],
                                        [0, 0, 1, 0],
                                        [0, 0, 0, 1]  ]
                                   )

aptd['fmri']['0,0,1,1']   = np.array([  [1, 0, 1, 0],
                                        [1, 0, 0, 1],
                                        [0, 1, 1, 0],
                                        [0, 1, 0, 1]  ]
                                   )

aptd['fmri']['0,1,1,0']   = np.array([  [1, 0, 0, 1],
                                        [0, 1, 0, 1],
                                        [1, 0, 1, 0],
                                        [0, 1, 1, 0]  ]
                                   )

Record the activity patterns in the output layer for each performance list. 

In [ ]:
aptd['behavior']          = {}

aptd['behavior']['0,0']   = np.array([  [1, 0, 0, 0],
                                        [1, 0, 0, 0],
                                        [0, 1, 0, 0],
                                        [0, 1, 0, 0]  ]
                                  )

aptd['behavior']['0,1']   = np.array([  [0, 0, 1, 0],
                                        [0, 0, 1, 0],
                                        [0, 0, 0, 1],
                                        [0, 0, 0, 1]  ]
                                  )

aptd['behavior']['1,0']   = np.array([  [1, 0, 0, 0],
                                        [0, 1, 0, 0],
                                        [1, 0, 0, 0],
                                        [0, 1, 0, 0]  ]
                                  )

aptd['behavior']['1,1']   = np.array([  [0, 0, 1, 0],
                                        [0, 0, 0, 1],
                                        [0, 0, 1, 0],
                                        [0, 0, 0, 1]  ]
                                  )


aptd['behavior']['0,0,1,1']= np.array([ [1, 0, 1, 0],
                                        [1, 1, 1, 1],
                                        [1, 1, 1, 1],
                                        [0, 1, 0, 1]  ]
                                   )

aptd['behavior']['0,1,1,0']= np.array([ [1, 0, 1, 0],
                                        [1, 1, 1, 1],
                                        [1, 1, 1, 1],
                                        [0, 1, 0, 1]  ]
                                   )



Record the labels for each performance list.


In [ ]:
aptd['instruction']          = {}

aptd['instruction']['0,0']   = np.array([ [1, 0, 0, 0],
                                          [1, 0, 0, 0],
                                          [0, 1, 0, 0],
                                          [0, 1, 0, 0]  ]
                                   )

aptd['instruction']['0,1']   = np.array([ [0, 0, 1, 0],
                                          [0, 0, 1, 0],
                                          [0, 0, 0, 1],
                                          [0, 0, 0, 1]  ]
                                   )

aptd['instruction']['1,0']   = np.array([ [1, 0, 0, 0],
                                          [0, 1, 0, 0],
                                          [1, 0, 0, 0],
                                          [0, 1, 0, 0]  ]
                                   )

aptd['instruction']['1,1']   = np.array([ [0, 0, 1, 0],
                                          [0, 0, 0, 1],
                                          [0, 0, 1, 0],
                                          [0, 0, 0, 1]  ]
                                   )

aptd['instruction']['0,0,1,1']=np.array([ [1, 0, 1, 0],
                                          [1, 0, 0, 1],
                                          [0, 1, 1, 0],
                                          [0, 1, 0, 1]  ]
                                   )

aptd['instruction']['0,1,1,0']=np.array([ [1, 0, 1, 0],
                                          [0, 1, 1, 0],
                                          [1, 0, 0, 1],
                                          [0, 1, 0, 1]  ]
                                   )

# Application: compare representations

### simple mean representations

*What does simple mean representation mean?*  The "mean representation" of a task is the mean hidden activation pattern, averaging over all stimuli.

In [ ]:
# compare two tasks which both use input dimension 0
aptfuns.aptd_ln_pl0_pl1_distofmeanreps(aptd,'fmri',[[0,0]],[[0,1]])

0.0

In [ ]:
# compare two tasks using two different input dimensions
aptfuns.aptd_ln_pl0_pl1_distofmeanreps(aptd,'fmri',[[0,0]],[[1,1]])

1.0

### featurewise-centered representations

*What does featurewise-centered representation mean?* Fix an input dimension (say 0).  Each tuple of feature values in the extraneous input dimension(s) then determines a map 

$$ \{\text{feature values in the relevant input dim} \} \longrightarrow \{\text{hidden activation patterns}\} $$

The *featurewise centered* representation is obtained by adding a constant offset to each map so that its values center around zero.  The representation itself is indexed family of all these feature-wise centered maps (indexed, that is, by tuples of feature values in the irrelevant dimensions).

In [ ]:
# while the network is multitasking
aptfuns.aptd_ln_pl0_pl1_id__featurewisecenterrepdist(aptd, 'fmri', [[0,0],[1,1]], [[0,1],[1,0]], 0)

2.0

In [ ]:
# while the network is single-tasking
aptfuns.aptd_ln_pl0_pl1_id__featurewisecenterrepdist(aptd, 'fmri', [[0,0]], [[0,1]], 0)

0.0

### beta representations

*What does 'beta representation' mean?*  The (the beta values of the) linear regressors with

* **depenent variables** = hidden activation patterns 
* **independent variables** = the feature value of input dimension 0 and 
* **controling for** the features in all other input dimensions

In [ ]:
# while the network is multitasking
aptfuns.aptd_ln_pl0_pl1_id__betarepdist(aptd, 'fmri', [[0,0],[1,1]], [[0,1],[1,0]], 0)

1.4142135623730951

In [ ]:
# while the network is single-tasking
aptfuns.aptd_ln_pl0_pl1_id__betarepdist(aptd, 'fmri', [[0,0]], [[0,1]], 0)

0.0

In [ ]:
# CHECK THAT THE FIRST NUMBER MAKES SENSE (IT DOES)
print(aptfuns.aptd_ln_pl_id__betarep(aptd, 'fmri', [[0,0],[1,1]], 0))
print(aptfuns.aptd_ln_pl_id__betarep(aptd, 'fmri', [[0,1],[1,0]], 0))
print(2**(1/2))

[[ 0.5 -0.5  0.   0. ]
 [-0.5  0.5  0.   0. ]]
[[ 0.   0.  -0.5  0.5]
 [ 0.   0.   0.5 -0.5]]
1.4142135623730951


# compute interaction effects

In [ ]:
# aggregate interaction effect between two given input dimensions while mapping two given output dimensions

np.mean(aptfuns.aptd_ln_id01_od01_plc__iena(aptd, 'fmri', 0, 1, 0, 1, 2))

2.0

In [ ]:
# aggregate interaction effect between two given input dimensions while mapping two given output dimensions

np.mean(aptfuns.aptd_ln_id01_od01_plc__iena(aptd, 'fmri', 0, 1, 0, 1, 1))

0.0